# 데이터 탐색 및 DB 연결 테스트

이 노트북은 가격 예측 프로젝트의 데이터 탐색과 데이터베이스 연결을 테스트합니다.

## 1. 라이브러리 임포트

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime

## 2. 데이터베이스 연결 테스트

In [ ]:
from src.data.db_connector import DatabaseConnector, SessionManager

# 데이터베이스 연결 테스트
print("=" * 50)
print("데이터베이스 연결 테스트")
print("=" * 50)

try:
    with DatabaseConnector() as db:
        print("✓ 데이터베이스 연결 성공!\n")
        
        # 테이블 목록 조회
        tables = db.get_table_list()
        print(f"✓ 사용 가능한 테이블 수: {len(tables)}")
        print("\n테이블 목록:")
        for i, table in enumerate(tables[:10], 1):
            print(f"  {i}. {table}")
        
        if len(tables) > 10:
            print(f"  ... 외 {len(tables) - 10}개")
        
        print("\n" + "=" * 50)
        print("연결 테스트 완료")
        print("=" * 50)
        
except Exception as e:
    print(f"✗ 연결 실패: {e}")

## 3. Step 1: 종목 필터링 (마스터 리스트 생성)

목표: NYSE, NASDAQ, AMEX 종목 중 조건을 만족하는 종목 추출

In [ ]:
# ORM 모델 임포트
from src.data.models import (
    DS2CtryQtInfo, 
    DS2Exchange, 
    DS2PrimQtPrc,
    VwDs2Pricing,
    VwDs2MktCap,
    RKDFndInfo  # Ticker, ISIN 정보를 위해 추가
)
from sqlalchemy import func, and_, or_
from sqlalchemy.orm import aliased

print("✓ ORM 모델 임포트 완료")

In [ ]:
# Step 1: NYSE, NASDAQ, AMEX 종목 리스트 조회 (ORM 사용 + Ticker/ISIN + 거래소명)
print("=" * 80)
print("Step 1: 종목 리스트 조회 (ORM) with Ticker/ISIN/Exchange")
print("=" * 80)

try:
    with DatabaseConnector() as db:
        with SessionManager(db) as session:
            # 1. NYSE, NASDAQ, AMEX 거래소 필터링
            print("\n[1] NYSE, NASDAQ, AMEX 필터링...")
            target_exchanges = session.query(DS2Exchange).filter(
                and_(
                    DS2Exchange.ExchCtryCode == 'US',
                    or_(
                        DS2Exchange.ExchName.like('%NYSE%'),
                        DS2Exchange.ExchName.like('%NASDAQ%'),
                        DS2Exchange.ExchName.like('%AMEX%')
                    )
                )
            ).all()
            
            print(f"✓ 대상 거래소:")
            for exch in target_exchanges[:5]:
                print(f"  - {exch.ExchName} (코드: {exch.ExchIntCode})")
            if len(target_exchanges) > 5:
                print(f"  ... 외 {len(target_exchanges) - 5}개")
            
            target_exch_codes = [exch.ExchIntCode for exch in target_exchanges]
            
            # 2. 종목 조회 (거래소, Ticker/ISIN 정보 포함)
            print(f"\n[2] 종목 조회 중...")
            
            # DS2PrimQtPrc에서 거래소별 종목 조회 후 DS2Exchange 조인
            stocks = session.query(
                DS2CtryQtInfo.InfoCode,
                DS2CtryQtInfo.DsCode,
                DS2CtryQtInfo.DsQtName,
                DS2CtryQtInfo.StatusCode,
                DS2CtryQtInfo.DelistDate,
                DS2PrimQtPrc.ExchIntCode,
                DS2Exchange.ExchName,
                DS2Exchange.ExchMnem,
                RKDFndInfo.Ticker,
                RKDFndInfo.ISIN,
                RKDFndInfo.Cusip,
                RKDFndInfo.Sedol,
            ).join(
                DS2PrimQtPrc,
                DS2CtryQtInfo.InfoCode == DS2PrimQtPrc.InfoCode
            ).join(
                DS2Exchange,
                DS2PrimQtPrc.ExchIntCode == DS2Exchange.ExchIntCode
            ).outerjoin(
                RKDFndInfo, 
                DS2CtryQtInfo.InfoCode == RKDFndInfo.Code
            ).filter(
                and_(
                    DS2CtryQtInfo.Region == 'US',
                    DS2CtryQtInfo.IsPrimQt == 1,
                    DS2PrimQtPrc.ExchIntCode.in_(target_exch_codes)
                )
            ).distinct().limit(100).all()
            
            print(f"✓ 조회된 종목 수: {len(stocks)} (샘플 100개)")
            
            # DataFrame으로 변환
            df_stocks = pd.DataFrame([
                {
                    'InfoCode': s.InfoCode,
                    'DsCode': s.DsCode,
                    'DsQtName': s.DsQtName,
                    'Exchange': s.ExchName,
                    'ExchMnem': s.ExchMnem,
                    'ExchIntCode': s.ExchIntCode,
                    'Ticker': s.Ticker,
                    'ISIN': s.ISIN,
                    'Cusip': s.Cusip,
                    'Sedol': s.Sedol,
                    'StatusCode': s.StatusCode,
                    'DelistDate': s.DelistDate,
                }
                for s in stocks
            ])
            
            print(f"\n✓ 데이터프레임 생성 완료:")
            print(f"  - Shape: {df_stocks.shape}")
            print(f"  - Ticker 매핑률: {df_stocks['Ticker'].notna().sum()}/{len(df_stocks)} ({df_stocks['Ticker'].notna().sum()/len(df_stocks)*100:.1f}%)")
            print(f"  - ISIN 매핑률: {df_stocks['ISIN'].notna().sum()}/{len(df_stocks)} ({df_stocks['ISIN'].notna().sum()/len(df_stocks)*100:.1f}%)")
            print(f"  - 상장폐지 종목: {df_stocks['DelistDate'].notna().sum()}")
            
            print(f"\n✓ 거래소별 분포:")
            print(df_stocks['Exchange'].value_counts())
            
            print(f"\n샘플 데이터 (처음 10개):")
            print(df_stocks[['DsCode', 'DsQtName', 'Exchange', 'Ticker', 'ISIN']].head(10))
            
except Exception as e:
    print(f"✗ 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    df_stocks = pd.DataFrame()

In [ ]:
df_stocks.to_csv("df_stocks.csv")

## 4. Step 2: 갭 상승 데이터 추출

목표: 거래일 기준으로 갭 상승 이벤트 찾기
- 갭 상승률 = (당일 시가 - 전일 종가) / 전일 종가 × 100

In [ ]:
# Step 2-1: 샘플 종목의 일별 가격 데이터 조회 및 갭 상승 계산
print("=" * 80)
print("Step 2: 갭 상승 데이터 추출")
print("=" * 80)

if not df_stocks.empty:
    # 샘플 종목 선택 (상장 중인 종목 중 5개)
    sample_stocks = df_stocks[df_stocks['DelistDate'].isna()].head(5)
    sample_info_codes = sample_stocks['InfoCode'].tolist()
    
    print(f"\n샘플 종목 ({len(sample_info_codes)}개):")
    for _, stock in sample_stocks.iterrows():
        print(f"  - {stock['DsQtName']} ({stock['Ticker']}) / {stock['Exchange']}")
    
    try:
        with DatabaseConnector() as db:
            with SessionManager(db) as session:
                print(f"\n일별 가격 데이터 조회 중...")
                
                # 일별 OHLCV 데이터 조회 (최근 2년)
                from datetime import datetime, timedelta
                start_date = datetime.now() - timedelta(days=730)  # 2년
                
                prices = session.query(
                    DS2PrimQtPrc.InfoCode,
                    DS2PrimQtPrc.MarketDate,
                    DS2PrimQtPrc.Open_,
                    DS2PrimQtPrc.High,
                    DS2PrimQtPrc.Low,
                    DS2PrimQtPrc.Close_,
                    DS2PrimQtPrc.Volume,
                ).filter(
                    and_(
                        DS2PrimQtPrc.InfoCode.in_(sample_info_codes),
                        DS2PrimQtPrc.MarketDate >= start_date,
                        DS2PrimQtPrc.Open_.isnot(None),
                        DS2PrimQtPrc.Close_.isnot(None)
                    )
                ).order_by(
                    DS2PrimQtPrc.InfoCode,
                    DS2PrimQtPrc.MarketDate
                ).all()
                
                print(f"✓ 조회된 레코드 수: {len(prices):,}")
                
                # DataFrame으로 변환
                df_prices = pd.DataFrame([
                    {
                        'InfoCode': p.InfoCode,
                        'MarketDate': p.MarketDate,
                        'Open': p.Open_,
                        'High': p.High,
                        'Low': p.Low,
                        'Close': p.Close_,
                        'Volume': p.Volume,
                    }
                    for p in prices
                ])
                
                # 종목별로 전일 종가 계산 및 갭 계산
                print(f"\n갭 상승 계산 중...")
                df_prices = df_prices.sort_values(['InfoCode', 'MarketDate'])
                df_prices['PrevClose'] = df_prices.groupby('InfoCode')['Close'].shift(1)
                df_prices['Gap_Pct'] = ((df_prices['Open'] - df_prices['PrevClose']) / df_prices['PrevClose'] * 100)
                df_prices['Intraday_Return'] = ((df_prices['Close'] - df_prices['Open']) / df_prices['Open'] * 100)
                
                # 갭 상승 필터링 (갭 > 0%)
                df_gap_up = df_prices[df_prices['Gap_Pct'] > 0].copy()
                df_gap_up['Target'] = (df_gap_up['Intraday_Return'] > 0).astype(int)
                
                print(f"✓ 갭 상승 이벤트 수: {len(df_gap_up):,}")
                print(f"✓ 전체 거래일 대비 비율: {len(df_gap_up)/len(df_prices)*100:.1f}%")
                
                # 종목 정보 매핑
                df_gap_up = df_gap_up.merge(
                    df_stocks[['InfoCode', 'DsQtName', 'Ticker', 'Exchange']],
                    on='InfoCode',
                    how='left'
                )
                
                print(f"\n✓ 갭 상승 통계:")
                print(f"  - 평균 갭 상승률: {df_gap_up['Gap_Pct'].mean():.2f}%")
                print(f"  - 중간 갭 상승률: {df_gap_up['Gap_Pct'].median():.2f}%")
                print(f"  - 최대 갭 상승률: {df_gap_up['Gap_Pct'].max():.2f}%")
                print(f"  - 당일 상승 비율 (Target=1): {df_gap_up['Target'].mean()*100:.1f}%")
                
                print(f"\n샘플 갭 상승 데이터 (최근 10개):")
                print(df_gap_up[['MarketDate', 'DsQtName', 'Ticker', 'Gap_Pct', 'Intraday_Return', 'Target']].tail(10))
                
    except Exception as e:
        print(f"✗ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        df_gap_up = pd.DataFrame()
else:
    print("\n✗ 종목 데이터가 없습니다.")
    df_gap_up = pd.DataFrame()

## 5. Step 3: 전체 파이프라인 테스트

**목표:** 
- DataLoader, DataPreprocessor, Feature Engineering 모듈 통합 테스트
- 소수 종목으로 빠르게 검증 후 전체 종목 적용

### 5-1. 모듈 임포트

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# 전체 파이프라인 모듈 임포트
from src.data.data_loader import DataLoader
from src.data.preprocessor import DataPreprocessor
from src.features import add_technical_features_parallel, add_market_context

print("✓ 전체 파이프라인 모듈 임포트 완료")

### 5-2. 데이터 로더 초기화 및 샘플 데이터 로드

**테스트 전략:**
- 먼저 10개 종목으로 빠르게 테스트
- 문제 없으면 전체 종목으로 확장

In [ ]:
# Step 1: DataLoader 초기화
print("="*80)
print("Step 1: DataLoader 초기화")
print("="*80)

loader = DataLoader(config_path="../config/config.yaml")
print(f"✓ Config loaded: {loader.config.get('data', {}).get('min_years', 10)} years")
print(f"✓ Batch size: {loader.config.get('data', {}).get('parallel', {}).get('batch_size', 100)}")

In [ ]:
# Step 2: 샘플 종목 리스트 조회 (10개)
print("\n" + "="*80)
print("Step 2: 샘플 종목 리스트 조회")
print("="*80)

# 전체 종목 리스트 로드
stock_list_full = loader.get_stock_list()
print(f"✓ 전체 종목 수: {len(stock_list_full)}")

# 샘플 10개 선택 (상장 중인 종목 우선)
sample_stocks = stock_list_full[stock_list_full['DelistDate'].isna()].head(10)
print(f"✓ 샘플 종목 수: {len(sample_stocks)}")

print("\n샘플 종목 목록:")
for idx, row in sample_stocks.iterrows():
    print(f"  {row['InfoCode']}: {row['DsQtName']} ({row['Ticker']}) - {row['Exchange']}")

In [ ]:
# Step 3: 섹터/산업 정보 로드
print("\n" + "="*80)
print("Step 3: 섹터/산업 정보 로드")
print("="*80)

sample_info_codes = sample_stocks['InfoCode'].tolist()
sector_info = loader.load_sector_industry(sample_info_codes)

# 샘플 종목과 병합
sample_stocks_with_sector = sample_stocks.merge(sector_info, on='InfoCode', how='left')

print(f"✓ 섹터 정보 조회 완료")
print(f"  - 섹터 매핑률: {sample_stocks_with_sector['sector'].notna().sum()}/{len(sample_stocks_with_sector)}")
print(f"  - 산업 매핑률: {sample_stocks_with_sector['industry'].notna().sum()}/{len(sample_stocks_with_sector)}")

print("\n섹터/산업 정보:")
print(sample_stocks_with_sector[['DsQtName', 'sector', 'industry']].head(10))

In [ ]:
# Step 4: OHLCV 데이터 로드 (최근 2년)
print("\n" + "="*80)
print("Step 4: OHLCV 데이터 로드")
print("="*80)

from datetime import datetime, timedelta

start_date = datetime.now() - timedelta(days=365 * 2)  # 2년
print(f"기간: {start_date.date()} ~ {datetime.now().date()}")

# 병렬로 OHLCV 로드 (수정: DataFrame 전달)
ohlcv_sample = loader.load_ohlcv_batch(sample_stocks_with_sector, start_date, adj_type=2)

print(f"\n✓ OHLCV 데이터 로드 완료")
print(f"  - 총 레코드 수: {len(ohlcv_sample):,}")
print(f"  - 종목 수: {ohlcv_sample['InfoCode'].nunique()}")
print(f"  - 날짜 범위: {ohlcv_sample['date'].min()} ~ {ohlcv_sample['date'].max()}")

print("\n샘플 데이터:")
print(ohlcv_sample.head())

In [ ]:
# Step 5: 시장 지수 데이터 로드 (SPY, QQQ, VIX)
print("\n" + "="*80)
print("Step 5: 시장 지수 데이터 로드")
print("="*80)

market_indices = loader.load_market_indices(start_date)

if not market_indices.empty:
    print(f"✓ 시장 지수 데이터 로드 완료")
    print(f"  - 레코드 수: {len(market_indices):,}")
    print(f"  - 컬럼: {list(market_indices.columns)}")
    print("\n샘플 데이터:")
    print(market_indices.head())
else:
    print("✗ 시장 지수 데이터 없음")

### 5-3. 데이터 전처리 (Preprocessor)

In [ ]:
# Step 6: 데이터 전처리 (병렬 처리)
print("\n" + "="*80)
print("Step 6: 데이터 전처리")
print("="*80)

# Preprocessor 초기화
preprocessor = DataPreprocessor(loader.config)

# preprocessed_df 생성 (병렬 처리 사용)
df_preprocessed = preprocessor.create_preprocessed_df(
    ohlcv_df=ohlcv_sample,
    stock_list=sample_stocks_with_sector,
    market_cap_df=None,  # 시가총액은 생략 (빠른 테스트용)
    use_parallel=True
)

print(f"\n✓ 전처리 완료")
print(f"  - 최종 Shape: {df_preprocessed.shape}")
print(f"  - 갭 상승 이벤트: {df_preprocessed['is_gap_up'].sum():,}")

### 5-4. Feature Engineering (기술적 지표 + 시장 컨텍스트)

In [ ]:
# Step 7: 기술적 지표 추가 (병렬 처리)
print("\n" + "="*80)
print("Step 7: 기술적 지표 추가")
print("="*80)

df_with_technical = add_technical_features_parallel(df_preprocessed, max_workers=4)

print(f"\n✓ 기술적 지표 추가 완료")
print(f"  - Shape: {df_with_technical.shape}")

# 추가된 컬럼 확인
technical_cols = ['ma_5', 'ma_20', 'ma_50', 'rsi_14', 'atr_14', 'bollinger_position']
print(f"\n기술적 지표 샘플:")
print(df_with_technical[['date', 'DsQtName', 'close'] + technical_cols].tail(5))

In [ ]:
# Step 8: 시장 컨텍스트 추가
print("\n" + "="*80)
print("Step 8: 시장 컨텍스트 추가")
print("="*80)

df_final = add_market_context(df_with_technical, market_indices)

print(f"\n✓ 시장 컨텍스트 추가 완료")
print(f"  - 최종 Shape: {df_final.shape}")

# 시장 컨텍스트 컬럼 확인
market_cols = ['spy_gap_pct', 'qqq_gap_pct', 'market_gap_diff', 'vix_level', 
               'sector', 'sector_gap_pct', 'sector_relative_gap']
print(f"\n시장 컨텍스트 샘플:")
print(df_final[['date', 'DsQtName', 'gap_pct'] + market_cols].tail(5))

### 5-5. 최종 결과 확인 및 통계

In [ ]:
# 최종 결과 확인
print("="*80)
print("최종 데이터프레임 정보")
print("="*80)

print(f"\n1. 기본 정보")
print(f"   - Shape: {df_final.shape}")
print(f"   - 종목 수: {df_final['InfoCode'].nunique()}")
print(f"   - 날짜 범위: {df_final['date'].min()} ~ {df_final['date'].max()}")
print(f"   - 메모리 사용량: {df_final.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print(f"\n2. 컬럼 구성 ({len(df_final.columns)}개)")
print(f"   - 전체 컬럼: {list(df_final.columns[:20])}...")

print(f"\n3. 갭 상승 이벤트 통계")
gap_up_df = df_final[df_final['is_gap_up'] == 1]
print(f"   - 갭 상승 이벤트 수: {len(gap_up_df):,}")
print(f"   - 평균 갭 상승률: {gap_up_df['gap_pct'].mean():.2f}%")
print(f"   - 당일 상승 비율: {gap_up_df['target_direction'].mean()*100:.1f}%")

print(f"\n4. Feature 커버리지")
print(f"   - 섹터 정보: {df_final['sector'].notna().sum()}/{len(df_final)} ({df_final['sector'].notna().mean()*100:.1f}%)")
print(f"   - 산업 정보: {df_final['industry'].notna().sum()}/{len(df_final)} ({df_final['industry'].notna().mean()*100:.1f}%)")
print(f"   - RSI 14: {df_final['rsi_14'].notna().sum()}/{len(df_final)} ({df_final['rsi_14'].notna().mean()*100:.1f}%)")
print(f"   - SPY 갭: {df_final['spy_gap_pct'].notna().sum()}/{len(df_final)} ({df_final['spy_gap_pct'].notna().mean()*100:.1f}%)")

print(f"\n5. 샘플 데이터 (갭 상승 이벤트만)")
display_cols = ['date', 'DsQtName', 'sector', 'gap_pct', 'target_direction', 
                'rsi_14', 'spy_gap_pct', 'market_gap_diff']
print(gap_up_df[display_cols].tail(10))

In [ ]:
df_final.to_csv("korea.csv")

### 5-6. 데이터 저장 (선택적)

In [ ]:
# 샘플 데이터 저장 (선택적)
# 주석 제거하여 실행

save_path = "../data/processed/sample_preprocessed_df.parquet"
preprocessor.save_to_file(df_final, save_path)
print(f"✓ 데이터 저장 완료: {save_path}")

print("샘플 테스트 완료! 문제 없으면 전체 종목으로 확장 가능합니다.")

## 6. 최종 데이터 추출 (5-1 ~ 5-4)

In [ ]:
import sys
from pathlib import Path
from datetime import datetime, timedelta

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from src.data.data_loader import DataLoader
from src.data.preprocessor import DataPreprocessor

print("✓ 모듈 임포트 완료")

# Step 1: 초기화
print("\n" + "="*80)
print("Step 1: 초기화")
print("="*80)

loader = DataLoader(config_path="../config/config.yaml")
preprocessor = DataPreprocessor(loader.config)
print("✓ DataLoader 및 Preprocessor 초기화 완료")

# Step 2: 전체 종목 리스트 로드
print("\n" + "="*80)
print("Step 2: 전체 종목 리스트 로드")
print("="*80)

stock_list_full = loader.get_stock_list()
print(f"✓ 전체 종목 수: {len(stock_list_full):,}")

# Step 3: 시장 지수 데이터 로드 (공통)
print("\n" + "="*80)
print("Step 3: 시장 지수 데이터 로드")
print("="*80)

start_date = datetime.now() - timedelta(days=365 * 20)  # 20년
market_indices = loader.load_market_indices(start_date)

print(f"✓ 시장 지수 데이터: {len(market_indices):,} 레코드")
print(f"✓ 날짜 범위: {market_indices['date'].min()} ~ {market_indices['date'].max()}")

# Step 4: 종목별 점진적 처리 (핵심!)
print("\n" + "="*80)
print("Step 4: 종목별 점진적 처리")
print("="*80)

df_final = preprocessor.process_stocks_incrementally(
    stock_list=stock_list_full,
    data_loader=loader,
    market_indices=market_indices,
    start_date=start_date,
    batch_size=100,  # 한 번에 100개 종목씩
    save_batch_files=True,
    batch_output_dir="../data/processed/batches",
    final_output="../data/processed",
    apply_outlier_fix=True
)

print(f"\n{'='*80}")
print("모든 처리 완료!")
print(f"{'='*80}")
print(f"최종 데이터 Shape: {df_final.shape}")
print(f"갭 상승 이벤트: {df_final['is_gap_up'].sum():,}")
print(f"파일 저장 위치: ../data/processed/preprocessed_df_full.parquet")

In [ ]:
#여기코드로 작성 

df_final = pd.read_parquet("../data/processed/preprocessed_df_full.parquet")
## 이상치 제거 
df_final.to_parquet("../data/processed/preprocessed_df_full_outlier_fix.parquet")


NameError: name 'pd' is not defined

---

## 다음 단계

### ✅ 완료된 것
1. **데이터 로더** - 종목 리스트, OHLCV, 섹터/산업, 시장 지수 로드
2. **전처리** - 갭 계산, 수익률, 거래량, 시간 Features (병렬 처리)
3. **기술적 지표** - MA, RSI, ATR, Bollinger Bands (병렬 처리)
4. **시장 컨텍스트** - SPY/QQQ/VIX 갭, 섹터별 갭

### 🔴 아직 구현 안 된 것
- 실적 발표 데이터 (earnings)
- 뉴스/센티먼트 데이터
- 애널리스트 레이팅 데이터
- 프리마켓 데이터 (Phase 2)

### 🚀 전체 종목 실행 방법
샘플 테스트가 성공하면, 위 코드에서:
- `sample_stocks = stock_list_full.head(10)` → `sample_stocks = stock_list_full`
- `start_date = datetime.now() - timedelta(days=365 * 2)` → `days=365 * 10` (10년)

### 📊 모델링 준비
- `df_final`을 사용하여 바로 모델 학습 가능
- 갭 상승 이벤트만 필터링: `df_final[df_final['is_gap_up'] == 1]`
- Train/Valid/Test 분할 후 모델 학습

## 99. [참고]QQQ/VIX 인덱스 확인

QQQ와 VIX 데이터를 못 가져온 이유를 확인합니다.

In [ ]:
# DB에 있는 인덱스 니모닉 확인
from src.data.models import DS2EquityIndex

print("="*80)
print("DB에 있는 인덱스 니모닉 확인")
print("="*80)

with DatabaseConnector() as db:
    with SessionManager(db) as session:
        # 1. NASDAQ 관련 인덱스 검색
        print("\n1. NASDAQ 관련 인덱스:")
        nasdaq_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%NASD%')
        ).limit(10).all()
        
        if nasdaq_indices:
            for idx in nasdaq_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ NASDAQ 관련 인덱스 없음")
        
        # 2. QQQ 관련 인덱스 검색
        print("\n2. QQQ 관련 인덱스:")
        qqq_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%QQQ%')
        ).limit(10).all()
        
        if qqq_indices:
            for idx in qqq_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ QQQ 관련 인덱스 없음")
        
        # 3. VIX 관련 인덱스 검색
        print("\n3. VIX 관련 인덱스:")
        vix_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%VIX%')
        ).limit(10).all()
        
        if vix_indices:
            for idx in vix_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ VIX 관련 인덱스 없음")
        
        # 4. CBOE 관련 인덱스 검색
        print("\n4. CBOE 관련 인덱스:")
        cboe_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%CBOE%')
        ).limit(10).all()
        
        if cboe_indices:
            for idx in cboe_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ CBOE 관련 인덱스 없음")
        
        # 5. S&P 관련 인덱스 확인 (성공한 것)
        print("\n5. S&P 관련 인덱스 (성공한 것):")
        sp_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%S&P%')
        ).limit(10).all()
        
        if sp_indices:
            for idx in sp_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ S&P 관련 인덱스 없음")
        
        # 6. 샘플로 처음 20개 인덱스 출력
        print("\n6. 샘플: DB의 처음 20개 인덱스")
        sample_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).limit(20).all()
        
        for idx in sample_indices:
            print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")

In [ ]:
# NASDAQ 100/Composite 인덱스 상세 검색
print("="*80)
print("NASDAQ 인덱스 상세 검색")
print("="*80)

with DatabaseConnector() as db:
    with SessionManager(db) as session:
        # 1. IndexDesc에서 NASDAQ 검색
        print("\n1. IndexDesc에 'NASDAQ' 포함:")
        nasdaq_all = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.IndexDesc.like('%NASDAQ%')
        ).limit(30).all()
        
        if nasdaq_all:
            for idx in nasdaq_all:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:12s} - {idx.IndexDesc}")
        else:
            print("   ❌ NASDAQ 관련 인덱스 없음")
        
        # 2. NDX 검색 (NASDAQ 100의 티커)
        print("\n2. NDX 검색:")
        ndx_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%NDX%')
        ).limit(10).all()
        
        if ndx_indices:
            for idx in ndx_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ NDX 관련 인덱스 없음")
        
        # 3. COMP 검색 (Composite)
        print("\n3. COMP 검색:")
        comp_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%COMP%')
        ).limit(15).all()
        
        if comp_indices:
            for idx in comp_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem} - {idx.IndexDesc}")
        else:
            print("   ❌ COMP 관련 인덱스 없음")

## 100. [참고] 한국 주식 인덱스 확인

KOSPI, KOSDAQ, KRX 등 한국 주식 관련 인덱스를 확인합니다.

In [ ]:
# DB에 있는 인덱스 니모닉 확인
from src.data.models import DS2EquityIndex, DS2Exchange, DS2CtryQtInfo, DS2PrimQtPrc,RKDFndInfo
# 한국 주식 인덱스 검색
print("="*80)
print("DB에 있는 한국 주식 인덱스 확인")
print("="*80)

with DatabaseConnector() as db:
    with SessionManager(db) as session:
        # 1. KOSPI 관련 인덱스 검색
        print("\n1. KOSPI 관련 인덱스:")
        kospi_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%KOSPI%')
        ).limit(20).all()
        
        if kospi_indices:
            for idx in kospi_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:20s} - {idx.IndexDesc}")
        else:
            print("   ❌ KOSPI 관련 인덱스 없음")
        
        # 2. KOSDAQ 관련 인덱스 검색
        print("\n2. KOSDAQ 관련 인덱스:")
        kosdaq_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%KOS%')
        ).limit(20).all()
        
        if kosdaq_indices:
            for idx in kosdaq_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:20s} - {idx.IndexDesc}")
        else:
            print("   ❌ KOSDAQ 관련 인덱스 없음")
        
        # 3. KRX 관련 인덱스 검색
        print("\n3. KRX 관련 인덱스:")
        krx_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%KRX%')
        ).limit(20).all()
        
        if krx_indices:
            for idx in krx_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:20s} - {idx.IndexDesc}")
        else:
            print("   ❌ KRX 관련 인덱스 없음")
        
        # 4. KOREA/KOR 관련 인덱스 검색
        print("\n4. KOREA 관련 인덱스:")
        korea_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%KOR%')
        ).limit(20).all()
        
        if korea_indices:
            for idx in korea_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:20s} - {idx.IndexDesc}")
        else:
            print("   ❌ KOREA 관련 인덱스 없음")
        
        # 5. IndexDesc에서 Korea 검색 (설명에서 검색)
        print("\n5. IndexDesc에 'Korea' 포함:")
        korea_desc_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.IndexDesc.like('%Korea%')
        ).limit(30).all()
        
        if korea_desc_indices:
            for idx in korea_desc_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:20s} - {idx.IndexDesc}")
        else:
            print("   ❌ Korea 관련 인덱스 없음")
        
        # 6. Seoul/SEO 관련 인덱스 검색
        print("\n6. Seoul/SEO 관련 인덱스:")
        seoul_indices = session.query(
            DS2EquityIndex.DSIndexCode,
            DS2EquityIndex.DSIndexMnem,
            DS2EquityIndex.IndexDesc
        ).filter(
            DS2EquityIndex.DSIndexMnem.like('%SEO%')
        ).limit(20).all()
        
        if seoul_indices:
            for idx in seoul_indices:
                print(f"   {idx.DSIndexCode}: {idx.DSIndexMnem:20s} - {idx.IndexDesc}")
        else:
            print("   ❌ Seoul/SEO 관련 인덱스 없음")

In [ ]:
# 한국 거래소 및 종목 리스트 확인
print("="*80)
print("한국 거래소 및 종목 리스트 확인")
print("="*80)

with DatabaseConnector() as db:
    with SessionManager(db) as session:
        # 1. 한국 거래소 확인
        print("\n[1] 한국 거래소 목록:")
        korea_exchanges = session.query(DS2Exchange).filter(
            DS2Exchange.ExchCtryCode == 'KR'
        ).all()
        
        if korea_exchanges:
            for exch in korea_exchanges:
                print(f"   - {exch.ExchName} (코드: {exch.ExchIntCode}, Mnem: {exch.ExchMnem})")
            
            korea_exch_codes = [exch.ExchIntCode for exch in korea_exchanges]
            
            # 2. 한국 종목 리스트 조회 (샘플)
            print(f"\n[2] 한국 종목 조회 중...")
            
            korea_stocks = session.query(
                DS2CtryQtInfo.InfoCode,
                DS2CtryQtInfo.DsCode,
                DS2CtryQtInfo.DsQtName,
                DS2CtryQtInfo.StatusCode,
                DS2PrimQtPrc.ExchIntCode,
                DS2Exchange.ExchName,
                DS2Exchange.ExchMnem,
                RKDFndInfo.Ticker,
                RKDFndInfo.ISIN,
            ).join(
                DS2PrimQtPrc,
                DS2CtryQtInfo.InfoCode == DS2PrimQtPrc.InfoCode
            ).join(
                DS2Exchange,
                DS2PrimQtPrc.ExchIntCode == DS2Exchange.ExchIntCode
            ).outerjoin(
                RKDFndInfo, 
                DS2CtryQtInfo.InfoCode == RKDFndInfo.Code
            ).filter(
                and_(
                    DS2CtryQtInfo.Region == 'KR',
                    DS2CtryQtInfo.IsPrimQt == 1,
                    DS2PrimQtPrc.ExchIntCode.in_(korea_exch_codes)
                )
            ).distinct().limit(50).all()
            
            print(f"✓ 조회된 한국 종목 수: {len(korea_stocks)} (샘플 50개)")
            
            # DataFrame으로 변환
            df_korea_stocks = pd.DataFrame([
                {
                    'InfoCode': s.InfoCode,
                    'DsCode': s.DsCode,
                    'DsQtName': s.DsQtName,
                    'Exchange': s.ExchName,
                    'ExchMnem': s.ExchMnem,
                    'Ticker': s.Ticker,
                    'ISIN': s.ISIN,
                    'StatusCode': s.StatusCode,
                }
                for s in korea_stocks
            ])
            
            print(f"\n✓ 한국 종목 데이터프레임:")
            print(f"  - Shape: {df_korea_stocks.shape}")
            if len(df_korea_stocks) > 0:
                print(f"  - Ticker 매핑률: {df_korea_stocks['Ticker'].notna().sum()}/{len(df_korea_stocks)}")
                print(f"  - ISIN 매핑률: {df_korea_stocks['ISIN'].notna().sum()}/{len(df_korea_stocks)}")
            
            print("\n샘플 데이터:")
            display(df_korea_stocks.head(20))
        else:
            print("   ❌ 한국 거래소 정보 없음")

## 이상치 제거 (IQR 방법)

DB 점검 중이므로 기존 parquet 파일에서 읽어서 IQR 이상치 제거 적용

In [2]:
import pandas as pd

# 1. 기존 데이터 로드
print("=" * 80)
print("IQR 이상치 제거")
print("=" * 80)

df_final = pd.read_parquet("../data/processed/preprocessed_df_full.parquet")
print(f"\n✓ 데이터 로드 완료")
print(f"  - Shape: {df_final.shape}")
print(f"  - 날짜 범위: {df_final['date'].min()} ~ {df_final['date'].max()}")
print(f"  - 종목 수: {df_final['InfoCode'].nunique():,}")

IQR 이상치 제거

✓ 데이터 로드 완료
  - Shape: (151825, 68)
  - 날짜 범위: 2006-01-17 00:00:00 ~ 2026-01-08 00:00:00
  - 종목 수: 2,250


In [3]:
# 2. IQR 이상치 제거 함수 정의
def remove_outliers_iqr(df, columns, multiplier=1.5, verbose=True):
    """
    IQR (Interquartile Range) 방법으로 이상치 제거
    
    공식:
    - Q1 = 25th percentile
    - Q3 = 75th percentile
    - IQR = Q3 - Q1
    - Lower Bound = Q1 - multiplier * IQR
    - Upper Bound = Q3 + multiplier * IQR
    """
    # 실제로 존재하는 컬럼만 선택
    columns = [c for c in columns if c in df.columns]
    
    if not columns:
        print("   ⚠ IQR 적용 가능한 컬럼이 없습니다.")
        return df
    
    initial_count = len(df)
    df_clean = df.copy()
    
    if verbose:
        print(f"\n=== IQR 이상치 제거 (multiplier={multiplier}) ===")
        print(f"적용 컬럼: {len(columns)}개")
    
    # 각 컬럼별로 이상치 마스크 생성
    outlier_mask = pd.Series(False, index=df_clean.index)
    col_stats = []
    
    for col in columns:
        if df_clean[col].isna().all():
            continue
            
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR
        
        # 이상치 판별
        col_outliers = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        outlier_count = col_outliers.sum()
        
        if outlier_count > 0:
            col_stats.append({
                'column': col,
                'Q1': Q1,
                'Q3': Q3,
                'IQR': IQR,
                'lower': lower_bound,
                'upper': upper_bound,
                'outliers': outlier_count,
                'pct': outlier_count / len(df_clean) * 100
            })
        
        # 전체 이상치 마스크에 합치기
        outlier_mask = outlier_mask | col_outliers
    
    # 이상치 제거
    df_clean = df_clean[~outlier_mask].copy()
    
    final_count = len(df_clean)
    removed_count = initial_count - final_count
    
    if verbose:
        print(f"\n컬럼별 이상치 통계:")
        for stat in sorted(col_stats, key=lambda x: x['outliers'], reverse=True)[:10]:
            print(f"  - {stat['column']}: {stat['outliers']:,}개 ({stat['pct']:.2f}%) "
                  f"[범위: {stat['lower']:.2f} ~ {stat['upper']:.2f}]")
        
        if len(col_stats) > 10:
            print(f"  ... 외 {len(col_stats) - 10}개 컬럼")
        
        print(f"\n✓ 이상치 제거: {initial_count:,} → {final_count:,} "
              f"({removed_count:,}개 제거, {removed_count/initial_count*100:.2f}%)")
    
    return df_clean

print("✓ IQR 이상치 제거 함수 정의 완료")

✓ IQR 이상치 제거 함수 정의 완료


In [4]:
# 3. 이상치 제거 대상 컬럼 정의 (연속형 Feature + Target)
CONTINUOUS_FEATURE_COLS = [
    # 갭 관련
    'gap_pct',
    
    # 전일 패턴
    'prev_return', 'prev_range_pct', 'prev_upper_shadow', 'prev_lower_shadow',
    
    # 거래량
    'volume_ratio',
    
    # 기술적 지표
    'rsi_14', 'atr_14', 'atr_ratio', 'bollinger_position',
    'return_5d', 'return_20d',
    
    # 시장 컨텍스트
    'market_gap_diff',
]

TARGET_COLS = [
    'target_return', 'target_max_return'
]

outlier_columns = CONTINUOUS_FEATURE_COLS + TARGET_COLS
print(f"이상치 제거 대상 컬럼: {len(outlier_columns)}개")
print(f"  - Feature: {len(CONTINUOUS_FEATURE_COLS)}개")
print(f"  - Target: {len(TARGET_COLS)}개")

이상치 제거 대상 컬럼: 15개
  - Feature: 13개
  - Target: 2개


In [5]:
# 4. 이상치 현황 확인 (제거 전)
print("=" * 80)
print("이상치 현황 (제거 전)")
print("=" * 80)

outlier_summary = []
for col in outlier_columns:
    if col not in df_final.columns or df_final[col].isna().all():
        continue
    
    Q1 = df_final[col].quantile(0.25)
    Q3 = df_final[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = ((df_final[col] < lower) | (df_final[col] > upper)).sum()
    
    outlier_summary.append({
        'column': col,
        'Q1': Q1,
        'median': df_final[col].median(),
        'Q3': Q3,
        'IQR': IQR,
        'lower_bound': lower,
        'upper_bound': upper,
        'min': df_final[col].min(),
        'max': df_final[col].max(),
        'outliers': outliers,
        'outlier_pct': outliers / len(df_final) * 100,
    })

df_outlier_summary = pd.DataFrame(outlier_summary).sort_values('outlier_pct', ascending=False)
print(f"\n총 이상치 현황:")
display(df_outlier_summary)

이상치 현황 (제거 전)

총 이상치 현황:


,column,Q1,median,Q3,IQR,lower_bound,upper_bound,min,max,outliers,outlier_pct
5,volume_ratio,0.493929,0.876955,1.701204,1.207275,-1.316983,3.512116,0.000002,19.545603,14194,9.348921
0,gap_pct,3.508772,4.376083,6.373293,2.864521,-0.788009,10.670074,3.000000,20.000000,13368,8.804874
12,market_gap_diff,2.962252,4.285297,6.535363,3.573111,-2.397414,11.895030,-9.231893,27.310934,12137,7.994072
7,atr_14,442.304025,888.295549,1913.912723,1471.608698,-1765.109021,4121.325769,1.674620,96205.501025,11258,7.415116
1,prev_return,-2.307692,0.652529,4.736842,7.044534,-12.874494,15.303644,-70.000000,206.060606,10001,6.587189
3,prev_upper_shadow,0.000000,0.013825,0.033333,0.033333,-0.050000,0.083333,0.000000,0.965079,8513,5.607113
13,target_return,-5.217391,-2.345416,1.480263,6.697654,-15.263873,11.526745,-66.666667,236.842105,8334,5.489215
4,prev_lower_shadow,0.003534,0.012195,0.024793,0.021260,-0.028356,0.056683,-0.061759,0.356061,7780,5.124321
8,atr_ratio,0.140915,0.225852,0.355345,0.214430,-0.180730,0.676989,0.000353,11.897822,7684,5.061090
11,return_20d,-42.105263,-0.917599,64.120061,106.225324,-201.443249,223.458047,-99.936316,18349.905482,6897,4.542730


In [6]:
# 5. IQR 이상치 제거 실행
print("=" * 80)
print("IQR 이상치 제거 실행")
print("=" * 80)

df_clean = remove_outliers_iqr(
    df_final, 
    columns=outlier_columns, 
    multiplier=1.5,  # 일반적인 이상치 제거 (더 관대하게 하려면 3.0 사용)
    verbose=True
)

print(f"\n최종 결과:")
print(f"  - 원본: {len(df_final):,}개")
print(f"  - 정제: {len(df_clean):,}개")
print(f"  - 제거됨: {len(df_final) - len(df_clean):,}개 ({(len(df_final) - len(df_clean))/len(df_final)*100:.2f}%)")

IQR 이상치 제거 실행

=== IQR 이상치 제거 (multiplier=1.5) ===
적용 컬럼: 15개

컬럼별 이상치 통계:
  - volume_ratio: 14,194개 (9.35%) [범위: -1.32 ~ 3.51]
  - gap_pct: 13,368개 (8.80%) [범위: -0.79 ~ 10.67]
  - market_gap_diff: 12,137개 (7.99%) [범위: -2.40 ~ 11.90]
  - atr_14: 11,258개 (7.42%) [범위: -1765.11 ~ 4121.33]
  - prev_return: 10,001개 (6.59%) [범위: -12.87 ~ 15.30]
  - prev_upper_shadow: 8,513개 (5.61%) [범위: -0.05 ~ 0.08]
  - target_return: 8,334개 (5.49%) [범위: -15.26 ~ 11.53]
  - prev_lower_shadow: 7,780개 (5.12%) [범위: -0.03 ~ 0.06]
  - atr_ratio: 7,684개 (5.06%) [범위: -0.18 ~ 0.68]
  - return_20d: 6,897개 (4.54%) [범위: -201.44 ~ 223.46]
  ... 외 4개 컬럼

✓ 이상치 제거: 151,825 → 88,699 (63,126개 제거, 41.58%)

최종 결과:
  - 원본: 151,825개
  - 정제: 88,699개
  - 제거됨: 63,126개 (41.58%)


In [7]:
# 6. 이상치 제거된 데이터 저장
output_path = "../data/processed/preprocessed_df_full_outlier_fix.parquet"

df_clean.to_parquet(output_path, index=False)

import os
file_size = os.path.getsize(output_path) / 1024 / 1024

print("=" * 80)
print("데이터 저장 완료")
print("=" * 80)
print(f"\n✓ 저장 경로: {output_path}")
print(f"✓ 파일 크기: {file_size:.2f} MB")
print(f"✓ 데이터 Shape: {df_clean.shape}")
print(f"✓ 날짜 범위: {df_clean['date'].min()} ~ {df_clean['date'].max()}")
print(f"✓ 종목 수: {df_clean['InfoCode'].nunique():,}")

데이터 저장 완료

✓ 저장 경로: ../data/processed/preprocessed_df_full_outlier_fix.parquet
✓ 파일 크기: 21.65 MB
✓ 데이터 Shape: (88699, 68)
✓ 날짜 범위: 2006-01-17 00:00:00 ~ 2026-01-08 00:00:00
✓ 종목 수: 2,225


In [8]:
# 7. 제거 전후 비교 (주요 컬럼)
print("=" * 80)
print("이상치 제거 전후 통계 비교")
print("=" * 80)

compare_cols = ['gap_pct', 'prev_return', 'target_return', 'target_max_return', 'volume_ratio']
compare_cols = [c for c in compare_cols if c in df_final.columns]

for col in compare_cols:
    print(f"\n[{col}]")
    print(f"  제거 전: min={df_final[col].min():.2f}, max={df_final[col].max():.2f}, mean={df_final[col].mean():.2f}, std={df_final[col].std():.2f}")
    print(f"  제거 후: min={df_clean[col].min():.2f}, max={df_clean[col].max():.2f}, mean={df_clean[col].mean():.2f}, std={df_clean[col].std():.2f}")

print("\n" + "=" * 80)
print("✓ 이상치 제거 완료!")
print("=" * 80)

이상치 제거 전후 통계 비교

[gap_pct]
  제거 전: min=3.00, max=20.00, mean=5.63, std=3.19
  제거 후: min=3.00, max=10.67, mean=4.62, std=1.64

[prev_return]
  제거 전: min=-70.00, max=206.06, mean=1.90, std=7.79
  제거 후: min=-12.87, max=15.30, mean=0.81, std=5.12

[target_return]
  제거 전: min=-66.67, max=236.84, mean=-1.62, std=6.93
  제거 후: min=-15.26, max=11.52, mean=-1.75, std=4.83

[target_max_return]
  제거 전: min=0.00, max=610.53, mean=4.56, std=5.75
  제거 후: min=0.00, max=16.76, mean=3.59, std=3.76

[volume_ratio]
  제거 전: min=0.00, max=19.55, mean=1.48, std=1.75
  제거 후: min=0.00, max=3.51, mean=0.94, std=0.70

✓ 이상치 제거 완료!
